In [1]:
# import library
import tensorflow as tf
import numpy as np
import sys, os,cv2
from scipy.misc import imread,imresize
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from imgaug import augmenters as iaa
import imgaug as ia
from scipy.ndimage import zoom
from sklearn.utils import shuffle
import matplotlib.animation as animation
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import make_blobs,make_moons,make_swiss_roll
from IPython.display  import IFrame

plt.style.use('seaborn')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
# create function for PCA using SVD
def pca_svd(data,return_project = False,mmax=1.5,mmin=0.5):
    data  = data - data.mean(0,keepdims=True)
    U,S,V = np.linalg.svd(data,full_matrices=False)
    projected_data = U @ np.diag(S) 
    V_T   = V.T
    s_range = (mmax-mmin) * (S-S.min())/(S.max()-S.min()) + mmin
    if not return_project: return V_T,s_range
    return projected_data,V_T,s_range
def tf_pca_svd(X,mmax=1.5,mmin=0.5,full=True): 
    s,U,V  = tf.linalg.svd(X,full_matrices=full)
    S      = tf.diag(s)
    s      = s.eval()
    ScaledS= (mmax-mmin)*((s-s.min())/(s.max()-s.min())) + mmin
    return U.eval(),S,V.eval(),ScaledS

In [3]:
# start the session
# sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

NameError: name 'sess' is not defined

In [ ]:
# make circle original data
from sklearn import  datasets
np.random.seed(6789)
n_samples   = 4000
data,label  = datasets.make_blobs(n_samples=n_samples,cluster_std=0.3,centers=2)
U,S,V,_     = tf_pca_svd(data)

print(data.shape)
print(U.shape)
print(S.shape)
print(V.shape)

In [10]:
# create sample data
from sklearn.datasets import make_blobs
data,label = make_blobs(9000,3)
print(data.shape)

(9000, 3)


In [11]:
# start with numpy svd
U,s,VT = np.linalg.svd(data,full_matrices=False)
print(U.shape)
print(s.shape)
print(VT.shape)

(9000, 3)
(3,)
(3, 3)


In [22]:
# get the gradient function
import numpy as anp
def T(x): return anp.swapaxes(x, -1, -2)
def grad_svd(usv, a,g):
    dot = anp.dot
    u = usv[0]
    s = usv[1]
    v = T(usv[2])

    m, n = a.shape[-2:]

    k = anp.min((m, n))
    # broadcastable identity array with shape (1, 1, ..., 1, k, k)
    i = anp.reshape(anp.eye(k), anp.concatenate((anp.ones(a.ndim - 2, dtype=int), (k, k))))

    f = 1 / (s[..., anp.newaxis, :]**2 - s[..., :, anp.newaxis]**2 + i)

    gu = g[0]
    gs = g[1]
    gv = T(g[2])

    utgu = dot(T(u), gu)
    vtgv = dot(T(v), gv)
    i_minus_uut = (anp.reshape(anp.eye(m), anp.concatenate((anp.ones(a.ndim - 2, dtype=int), (m, m)))) - dot(u, T(u)))

    t1 = (f * (utgu - T(utgu))) * s[..., anp.newaxis, :]
    t1 = t1 + i * gs[..., :, anp.newaxis]
    t1 = t1 + s[..., :, anp.newaxis] * (f * (vtgv - T(vtgv)))

    t1 = dot(dot(u, t1), T(v))
    t1 = t1 + dot(i_minus_uut, dot(gu, T(v) / s[..., :, anp.newaxis]))
    return t1

S  = np.diag(s)
gu = np.ones_like(data) @ (S @ VT).T
gs = np.diag(U.T @ np.ones_like(data) @ VT.T)
gv = (U @ S).T @ np.ones_like(data)
grad_svd = grad_svd([U,s,VT],data,[gu,gs,gv])
print(grad_svd)


[[ 1.39547963e+05 -2.86660322e+03 -1.31819497e+03]
 [ 1.16524227e+04 -2.39364816e+02 -1.10070865e+02]
 [ 1.56829614e+04 -3.22160400e+02 -1.48144053e+02]]
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 ...
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


In [17]:
# use the high level api
from autograd import elementwise_grad as egrad
import autograd.numpy as nps
def get_svd(data):
    U,s,V = nps.linalg.svd(data,full_matrices=False)
    S = nps.diag(s)
    return U @ S @ V.T

grad_ssvd = egrad(get_svd)
print(grad_ssvd(data).shape)
print(grad_ssvd(data).sum())
print(grad_ssvd(data).mean())

7sss890
7890
(4000, 2)
7sss890
7890
7999.999999999996
7sss890
7890
0.9999999999999996
